# Recommendation Flow Example

## Step 1: Loading Data

In [1]:
from models.csv_loader import CSVLoader
from models.products.product_registry import ProductRegistry
from models.products.product_mapping_row import ProductMappingRow
from models.products.product_row import ProductRow

product_registry = ProductRegistry(CSVLoader(ProductRow).read(), CSVLoader(ProductMappingRow).read())

In [2]:
from models.users.user_registry import UserRegistry
from models.users.user_mapping_row import UserMappingRow
from models.users.user_row import UserRow

user_registry = UserRegistry(CSVLoader(UserRow).read(), CSVLoader(UserMappingRow).read())

In [3]:
from models.ratings.rating_registry import RatingRegistry
from models.ratings.rating_row import RatingRow

rating_registry = RatingRegistry(CSVLoader(RatingRow).read(), user_registry, product_registry)

## Step 2: Recommendation

In [4]:
from recommendation.recommenders.random_recommender import RandomRecommender
from recommendation.recommenders.ilp_recommender import ILPRecommender

#recommender = RandomRecommender(product_registry, user_registry, rating_registry)
recommender = ILPRecommender(product_registry, user_registry, rating_registry)

rules [{'user': ['25-34'], 'product': ['Comedy']}, {'user': [], 'product': ['Crime']}, {'user': ['F'], 'product': []}, {'user': ['25-34'], 'product': ['Action']}, {'user': ['45-49'], 'product': []}, {'user': ['18-24'], 'product': ['Drama']}, {'user': [], 'product': ['Horror']}, {'user': ['25-34', 'F'], 'product': []}, {'user': [], 'product': ['Adventure']}, {'user': ['M'], 'product': ['Drama']}]


In [5]:
from models.ratings.rating import Rating
from models.users.user import User
import time

target_user = user_registry.add_user(User(eid=0, uid=0, gender="F", age="18-24"))

movie1 = product_registry.find_by_pid(pid=2346) # Fish Called Wanda, A (1988)
movie2 = product_registry.find_by_pid(pid=1762) # Fast Times at Ridgemont High (1982)

rating_registry.add_rating(Rating(target_user, movie1, 4, int(time.time())))
rating_registry.add_rating(Rating(target_user, movie2, 3, int(time.time())))

Rating(user=User(eid=-1, uid=6041, gender='F', age='18-24'), product=Product(eid=616, pid=1762, name='Deep Rising (1998)', genre='Action'), rating=3, timestamp=1725457045)

In [6]:
recommendation_paths = recommender.recommend(target_user)
recommendation_paths

[RecoPath(nodes=[RecoNode(type='user', entity_id=-1), RecoNode(type='product', entity_id=23)], rels=[RecoRel(in_node=RecoNode(type='product', entity_id=23), relation='Adventure', out_node=RecoNode(type='product', entity_id=23))])]

## Step 3: Explanation

In [7]:
from recommendation.explainers.llm_explainer import LLMExplainer

# repo_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
repo_id = "meta-llama/Meta-Llama-3-8B-Instruct"
# repo_id = "google/gemma-7b"
# repo_id = "mistralai/Mistral-7B-Instruct-v0.2"

explainer = LLMExplainer(product_registry, user_registry, rating_registry, repo_id)

In [8]:
facts = explainer.generate_facts(recommendation_paths[0])
print(facts)

AttributeError: 'LLMExplainer' object has no attribute 'generate_facts'

In [9]:
explanation = explainer.explain(recommendation_paths[0])
print(explanation)

StopIteration: 